# 1. Install and Import Dependencies

In [ ]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# 2. Instantiate Model

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

# 3. Encode and Calculate Sentiment

In [5]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [6]:
result = model(tokens)

In [7]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [8]:
int(torch.argmax(result.logits))+1

4

# 4. Collect Reviews

In [18]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont?start=1')
soup = BeautifulSoup(r.text, 'html.parser')

In [ ]:
soup

In [20]:
results = soup.find_all('p', class_='comment__09f24__D0cxf')
reviews = [result.text for result in results]

In [21]:
reviews

["Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!",
 "Six of us met here for breakfast before our walk to Manly. We were enjoying visiting with each other so much that I apologize for not taking any photos. We all enjoyed our food, as well as our coffee and tea drinks.We were greeted immediately by a friendly server asking if we would like to sit inside or out. We said we would like inside, but weren't exactly sure how many were joining us yet- at least 4. 

# 5. Load Reviews into DataFrame and Score

In [22]:
import numpy as np
import pandas as pd

In [23]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [24]:
df

,review
0,Very cute coffee shop and restaurant. They hav...
1,Six of us met here for breakfast before our wa...
2,Great place with delicious food and friendly s...
3,Great food amazing coffee and tea. Short walk ...
4,"Great service, lovely location, and really ama..."
5,Ricotta hot cakes! These were so yummy. I ate ...
6,It was ok. Had coffee with my friends. I'm new...
7,We came for brunch twice in our week-long visi...
8,I came to Social brew cafe for brunch while ex...
9,Great staff and food. Must try is the pan fri...


In [25]:
df['review'].iloc[0]

"Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!"

In [26]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [27]:
sentiment_score(df['review'].iloc[1])

4

In [28]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [29]:
df

,review,sentiment
0,Very cute coffee shop and restaurant. They hav...,4
1,Six of us met here for breakfast before our wa...,4
2,Great place with delicious food and friendly s...,5
3,Great food amazing coffee and tea. Short walk ...,5
4,"Great service, lovely location, and really ama...",5
5,Ricotta hot cakes! These were so yummy. I ate ...,5
6,It was ok. Had coffee with my friends. I'm new...,3
7,We came for brunch twice in our week-long visi...,4
8,I came to Social brew cafe for brunch while ex...,5
9,Great staff and food. Must try is the pan fri...,5


In [30]:
df['review'].iloc[3]

'Great food amazing coffee and tea. Short walk from the harbor. Staff was very friendly'